# Mode detection in MOs test

## Test 1: UniDip

### A: On a classic bimodal Gaussian

In [ ]:
%matplotlib widget
import numpy as np
from numpy.random import default_rng
import matplotlib.pyplot as plt
from qcnico import plt_utils


rng = default_rng()
X = np.sort(np.hstack([rng.standard_normal(300)-3, rng.standard_normal(300)+3]))

fig, ax = plt.subplots()

plt_utils.histogram(X,nbins=50,plt_objs=(fig,ax))




In [ ]:
from unidip import UniDip

intervals = UniDip(X).run()
print(intervals)

for k,I in enumerate(intervals):
    print(f" Interval # {k+1} [{X[I[0]]}; {X[I[1]]}]")

## Step 2: Examining density distribution for various MOs

First, let's load all of the data

In [ ]:
%matplotlib widget

from os import path
from qcnico.coords_io import read_xsf
from qcnico.remove_dangling_carbons import remove_dangling_carbons


datadir = path.expanduser("~/Desktop/simulation_outputs/percolation/40x40")
posdir = path.join(datadir, "structures")

lbls = [f'bigMAC-{n}' for n in [2,3,5,6,7,9,10]]

rCC_MAC = 1.8

def getARPACKdata(datadir,lbls):
    edir = path.join(datadir,'eARPACK')
    Mdir = path.join(datadir,'MOs_ARPACK')
    ee = [np.load(path.join(edir,f'eARPACK_{lbl}.npy')) for lbl in lbls]
    MM = [np.load(path.join(Mdir,f'MOs_ARPACK_{lbl}.npy')) for lbl in lbls]
    return ee, MM

def get_pos(posdir,lbls):
    return [remove_dangling_carbons(read_xsf(path.join(posdir,f"{lbl}_relaxed.xsf"))[0],rCC_MAC) for lbl in lbls]
    


ee, MM = getARPACKdata(datadir,lbls)
posarrs = get_pos(posdir,lbls)

Pick a specific MO based (previoulsy plotted in `santity_check.ipynb`) that we know is multimodal

In [ ]:
from qcnico.qcplots import plot_MO

istruc = 3 
iMO = -1 

pos = posarrs[istruc]
M = MM[istruc]

plot_MO(pos,M,iMO,dotsize=1.0,show_COM=True,show_rgyr=True,show=True,usetex=False)

In [ ]:
psi = np.abs(M[:,istruc])
plt_utils.histogram(psi,nbins=100,log_counts=False)
print(np.min(psi))
print((psi>1e-2).sum())

In [ ]:
from qcnico.qchemMAC import gridify_MO

X = np.sort(pos[:,0])
Y = np.sort(pos[:,1])

deltaX = X[-1] - X[0]
deltaY = Y[-1] - Y[0]

res = 10.0 #resolution of grid in angstroms

xgrid = np.linspace(X[0]-deltaX*res, X[-1]+deltaX*res, int(deltaX/res)+1)
ygrid = np.linspace(Y[0]-deltaY*res, Y[-1]+deltaY*res, int(deltaY/res)+1)


XX, YY = np.meshgrid(xgrid,ygrid,sparse=True)

rho = gridify_MO(pos, M, iMO, XX, YY,eps=1e-3)

print(type(rho)) 
plt.imshow(rho, cmap='plasma')
plt.show()

In [ ]:
rho.nonzero()